In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("Main_data.txt" , header=None , sep="\n")
data

,0
0,>2BBV:C
1,TQTAPVPQQNVPALTRLSQPGLAFLKCAFAPPDFNTDPGKGIPDRF...
2,0000000000001110001001000100000001000000000000...
3,>1A9N:D
4,IRPNHTIYINNMNDKIKKEELKRSLYALFSQFGHVVDIVALKTMKM...
...,...
202,MVVTRIAPSPTGDPHVGTAYIALFNYAWARRNGGRFIVRIEDTDRA...
203,0000001010000000000000000000000000000000101100...
204,>1QF6:A
205,PVITLPDGSQRHYDHAVSPMDVALDIGPGLAKACIAGRVNGELVDA...


In [3]:
s = "TQTAPVPQQNVPALTRLSQPGLAFLKCAFAPPDFNTDPGKGIPDRFEGKVVTRKDVLNQSINFTANRDTFILIAPTPGVAYWVADVPAGTFPISTTTFNAVNFPGFNSMFGNAAASRSDQVSSFRYASMNVGIYPTSNLMQFAGSITVWKCPVKLSNVQFPVATTPATSALVHTLVGLDGVLAVGPDNFSESFIKGVFSQSVCNEPDFEFSDILEGIQTLPPANVTVATSGQPFNLAAGAEAVSGIVGWGNMDTIVIRVSAPTGAVNSAILKTWACLEYRPNPNAMLYQFGHDSPPCDEVALQEYRTVARSLPVAVIAAQN"
b = "000000000000111000100100010000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000"

In [4]:
num = 1
map = {}
for d in data[0] :
    if d[0] == ">":
        key = d[1:-4]
        if key in map :
            continue
        map[key] = num
        num += 1
map

{'2B': 1,
 '1A': 2,
 '1B': 3,
 '1C': 4,
 '1D': 5,
 '1E': 6,
 '1F': 7,
 '1G': 8,
 '1H': 9,
 '1I': 10,
 '1J': 11,
 '1K': 12,
 '1L': 13,
 '1M': 14,
 '1N': 15,
 '1Q': 16}

In [5]:
protein = []
alphabet = []
bind = []
for i in range(len(data[0])) :
    if data[0][i][0] == ">":
        key = data[0][i][1:-4]
        for j in range(len(data[0][i+1])) :
            protein.append(map[key])
            alphabet.append(ord(data[0][i+1][j]))
            bind.append(data[0][i+2][j])

In [6]:
x = pd.DataFrame()
x["protein"] = protein
x["alpha"] = alphabet
y = bind

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV , GridSearchCV , train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [8]:
x_train , x_test , y_train , y_test = train_test_split(x , y)

In [9]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
clf = SVC(C=0.001, gamma=0.01)
clf.fit(x_train_scaled , y_train)

SVC(C=0.001, gamma=0.01)

In [10]:
x_test_scaled = scaler.transform(x_test)
clf.score(x_test_scaled , y_test)

0.7401207464324918

In [11]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [158]:
params = {
    "gamma": [0.01 , 0.05],
    "C": [0.001 , 0.0008]
}
tempclf = GridSearchCV(SVC() , params , n_jobs=5 , verbose=2)
tempclf.fit(x_scaled , y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  20 out of  20 | elapsed:   22.2s finished


GridSearchCV(estimator=SVC(), n_jobs=5,
             param_grid={'C': [0.001, 0.0008], 'gamma': [0.01, 0.05]},
             verbose=2)

In [179]:
print(tempclf.best_estimator_ , tempclf.best_score_)

SVC(C=0.001, gamma=0.01) 0.7428983166378433


In [12]:
test = pd.read_csv("Test_data.txt" , header=None , sep="\n")

In [13]:
protein_test = []
alphabet_test = []
for i in range(len(test[0])) :
    if test[0][i][0] == ">":
        key = test[0][i][1:-4]
        for j in range(len(test[0][i+1])) :
            if key not in map.keys() :
                print("NEWW")
                map[key] = num
                num += 1
            protein_test.append(map[key])
            alphabet_test.append(ord(test[0][i+1][j]))

NEWW
NEWW
NEWW
NEWW


In [14]:
x_test = pd.DataFrame()
x_test["protein"] = protein_test
x_test["alpha"] = alphabet_test

In [15]:
x_test_scaled= scaler.transform(x_test)

In [16]:
clf = SVC(C=0.001, gamma=0.01)
clf.fit(x_scaled , y)
y_test = clf.predict(x_test_scaled)
y_test

array(['0', '0', '0', ..., '0', '0', '0'], dtype='<U1')

In [229]:
soln = pd.DataFrame()
soln["Id"] = [i for i in range(5497)]
soln["Expected"] = y_test
soln

,Id,Expected
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
5492,5492,0
5493,5493,0
5494,5494,0
5495,5495,0


In [17]:
for i in y_test:
    if i == "1":
        print("OOO")